<a href="https://colab.research.google.com/github/Kanha02052002/S8-Capstone/blob/main/S8_Capstone_JD_Gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q datasets accelerate torch transformers sentence-transformers scikit-learn hf_xet nltk matplotlib seaborn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.1/75.1 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.0/201.0 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.9/193.9 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 76.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.4/242.4 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.6/221.6 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.3/377.3 kB 33.7 MB/s eta 0:00:00


In [2]:
import os
import re
import pandas as pd
import numpy as np
from datasets import load_dataset
import nltk
from nltk.corpus import stopwords
import warnings
warnings.filterwarnings("ignore")
nltk.download('stopwords', quiet=True)

True

In [3]:
data=load_dataset("lof223/job_description")

README.md:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

job_descriptions.csv:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1615940 [00:00<?, ? examples/s]

In [4]:
data

DatasetDict({
    train: Dataset({
        features: ['Job Id', 'Experience', 'Qualifications', 'Salary Range', 'location', 'Country', 'latitude', 'longitude', 'Work Type', 'Company Size', 'Job Posting Date', 'Preference', 'Contact Person', 'Contact', 'Job Title', 'Role', 'Job Portal', 'Job Description', 'Benefits', 'skills', 'Responsibilities', 'Company', 'Company Profile'],
        num_rows: 1615940
    })
})

In [5]:
def _overview_dataset(data):
  data=data['train'].to_pandas()
  print(f"Dataset Type: {type(data)}\n")
  print(f"Dataset Length: {len(data)}\n")
  print(f"Dataset Features: {data.columns.values}")
  print(f"Dataset Features Types: {data.dtypes.values}")
  print(f"Dataset Head: {data.head()}")
  print(f"Dataset Description: {data.describe()}")
  print(f"Dataset Info: {data.info()}")
  print(f"Dataset Shape: {data.shape}")
  print(f"Dataset Memory Usage: {data.memory_usage()}")
  print(f"Dataset Null Values: {data.isnull().sum()}")
  print(f"Dataset Duplicated Values: {data.duplicated().sum()}")
  print(f"Dataset Unique Values: {data.nunique()}")


In [6]:
_overview_dataset(data)

Dataset Type: <class 'pandas.core.frame.DataFrame'>

Dataset Length: 1615940

Dataset Features: ['Job Id' 'Experience' 'Qualifications' 'Salary Range' 'location'
 'Country' 'latitude' 'longitude' 'Work Type' 'Company Size'
 'Job Posting Date' 'Preference' 'Contact Person' 'Contact' 'Job Title'
 'Role' 'Job Portal' 'Job Description' 'Benefits' 'skills'
 'Responsibilities' 'Company' 'Company Profile']
Dataset Features Types: [dtype('int64') dtype('O') dtype('O') dtype('O') dtype('O') dtype('O')
 dtype('float64') dtype('float64') dtype('O') dtype('int64') dtype('O')
 dtype('O') dtype('O') dtype('O') dtype('O') dtype('O') dtype('O')
 dtype('O') dtype('O') dtype('O') dtype('O') dtype('O') dtype('O')]
Dataset Head:              Job Id     Experience Qualifications Salary Range    location  \
0  1089843540111562  5 to 15 Years         M.Tech    $59K-$99K     Douglas   
1   398454096642776  2 to 12 Years            BCA   $56K-$116K    Ashgabat   
2   481640072963533  0 to 12 Years            P

In [7]:
data['train'][0]

{'Job Id': 1089843540111562,
 'Experience': '5 to 15 Years',
 'Qualifications': 'M.Tech',
 'Salary Range': '$59K-$99K',
 'location': 'Douglas',
 'Country': 'Isle of Man',
 'latitude': 54.2361,
 'longitude': -4.5481,
 'Work Type': 'Intern',
 'Company Size': 26801,
 'Job Posting Date': '2022-04-24',
 'Preference': 'Female',
 'Contact Person': 'Brandon Cunningham',
 'Contact': '001-381-930-7517x737',
 'Job Title': 'Digital Marketing Specialist',
 'Role': 'Social Media Manager',
 'Job Portal': 'Snagajob',
 'Job Description': 'Social Media Managers oversee an organizations social media presence. They create and schedule content, engage with followers, and analyze social media metrics to drive brand awareness and engagement.',
 'Benefits': "{'Flexible Spending Accounts (FSAs), Relocation Assistance, Legal Assistance, Employee Recognition Programs, Financial Counseling'}",
 'skills': 'Social media platforms (e.g., Facebook, Twitter, Instagram) Content creation and scheduling Social media anal

In [8]:
data=data['train'].to_pandas()

In [9]:
data.head()

,Job Id,Experience,Qualifications,Salary Range,location,Country,latitude,longitude,Work Type,Company Size,...,Contact,Job Title,Role,Job Portal,Job Description,Benefits,skills,Responsibilities,Company,Company Profile
0,1089843540111562,5 to 15 Years,M.Tech,$59K-$99K,Douglas,Isle of Man,54.2361,-4.5481,Intern,26801,...,001-381-930-7517x737,Digital Marketing Specialist,Social Media Manager,Snagajob,Social Media Managers oversee an organizations...,"{'Flexible Spending Accounts (FSAs), Relocatio...","Social media platforms (e.g., Facebook, Twitte...","Manage and grow social media accounts, create ...",Icahn Enterprises,"{""Sector"":""Diversified"",""Industry"":""Diversifie..."
1,398454096642776,2 to 12 Years,BCA,$56K-$116K,Ashgabat,Turkmenistan,38.9697,59.5563,Intern,100340,...,461-509-4216,Web Developer,Frontend Web Developer,Idealist,Frontend Web Developers design and implement u...,"{'Health Insurance, Retirement Plans, Paid Tim...","HTML, CSS, JavaScript Frontend frameworks (e.g...","Design and code user interfaces for websites, ...",PNC Financial Services Group,"{""Sector"":""Financial Services"",""Industry"":""Com..."
2,481640072963533,0 to 12 Years,PhD,$61K-$104K,Macao,"Macao SAR, China",22.1987,113.5439,Temporary,84525,...,9687619505,Operations Manager,Quality Control Manager,Jobs2Careers,Quality Control Managers establish and enforce...,"{'Legal Assistance, Bonuses and Incentive Prog...",Quality control processes and methodologies St...,Establish and enforce quality control standard...,United Services Automobile Assn.,"{""Sector"":""Insurance"",""Industry"":""Insurance: P..."
3,688192671473044,4 to 11 Years,PhD,$65K-$91K,Porto-Novo,Benin,9.3077,2.3158,Full-Time,129896,...,+1-820-643-5431x47576,Network Engineer,Wireless Network Engineer,FlexJobs,"Wireless Network Engineers design, implement, ...","{'Transportation Benefits, Professional Develo...",Wireless network design and architecture Wi-Fi...,"Design, configure, and optimize wireless netwo...",Hess,"{""Sector"":""Energy"",""Industry"":""Mining, Crude-O..."
4,117057806156508,1 to 12 Years,MBA,$64K-$87K,Santiago,Chile,-35.6751,-71.5429,Intern,53944,...,343.975.4702x9340,Event Manager,Conference Manager,Jobs2Careers,A Conference Manager coordinates and manages c...,"{'Flexible Spending Accounts (FSAs), Relocatio...",Event planning Conference logistics Budget man...,Specialize in conference and convention planni...,Cairn Energy,"{""Sector"":""Energy"",""Industry"":""Energy - Oil & ..."


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1615940 entries, 0 to 1615939
Data columns (total 23 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   Job Id            1615940 non-null  int64  
 1   Experience        1615940 non-null  object 
 2   Qualifications    1615940 non-null  object 
 3   Salary Range      1615940 non-null  object 
 4   location          1615940 non-null  object 
 5   Country           1615940 non-null  object 
 6   latitude          1615940 non-null  float64
 7   longitude         1615940 non-null  float64
 8   Work Type         1615940 non-null  object 
 9   Company Size      1615940 non-null  int64  
 10  Job Posting Date  1615940 non-null  object 
 11  Preference        1615940 non-null  object 
 12  Contact Person    1615940 non-null  object 
 13  Contact           1615940 non-null  object 
 14  Job Title         1615940 non-null  object 
 15  Role              1615940 non-null  object 
 16  

In [11]:
data.columns

Index(['Job Id', 'Experience', 'Qualifications', 'Salary Range', 'location',
       'Country', 'latitude', 'longitude', 'Work Type', 'Company Size',
       'Job Posting Date', 'Preference', 'Contact Person', 'Contact',
       'Job Title', 'Role', 'Job Portal', 'Job Description', 'Benefits',
       'skills', 'Responsibilities', 'Company', 'Company Profile'],
      dtype='object')

In [12]:
data.columns=data.columns.str.lower()

In [13]:
# <Job Title, Skills, Experience, Work-Mode, Salary>
data=data[['job title','skills','experience','work type','salary range','job description']]
data.columns

Index(['job title', 'skills', 'experience', 'work type', 'salary range',
       'job description'],
      dtype='object')

In [14]:
data.head()

,job title,skills,experience,work type,salary range,job description
0,Digital Marketing Specialist,"Social media platforms (e.g., Facebook, Twitte...",5 to 15 Years,Intern,$59K-$99K,Social Media Managers oversee an organizations...
1,Web Developer,"HTML, CSS, JavaScript Frontend frameworks (e.g...",2 to 12 Years,Intern,$56K-$116K,Frontend Web Developers design and implement u...
2,Operations Manager,Quality control processes and methodologies St...,0 to 12 Years,Temporary,$61K-$104K,Quality Control Managers establish and enforce...
3,Network Engineer,Wireless network design and architecture Wi-Fi...,4 to 11 Years,Full-Time,$65K-$91K,"Wireless Network Engineers design, implement, ..."
4,Event Manager,Event planning Conference logistics Budget man...,1 to 12 Years,Intern,$64K-$87K,A Conference Manager coordinates and manages c...


In [15]:
def _check_column_types(data):
  for col in data.columns:
    print(f"{col}: {data[col].dtype}")

def _print_structured_data(count, data):
  for i in range(count):
    print(f"\n---------------------\nData-{i+1}\n")
    for col_name, value in data.iloc[i].items():
      print(f"{col_name}: {value}")

In [16]:
_check_column_types(data)

job title: object
skills: object
experience: object
work type: object
salary range: object
job description: object


In [17]:
_print_structured_data(5,data)


---------------------
Data-1

job title: Digital Marketing Specialist
skills: Social media platforms (e.g., Facebook, Twitter, Instagram) Content creation and scheduling Social media analytics and insights Community engagement Paid social advertising
experience: 5 to 15 Years
work type: Intern
salary range: $59K-$99K
job description: Social Media Managers oversee an organizations social media presence. They create and schedule content, engage with followers, and analyze social media metrics to drive brand awareness and engagement.

---------------------
Data-2

job title: Web Developer
skills: HTML, CSS, JavaScript Frontend frameworks (e.g., React, Angular) User experience (UX)
experience: 2 to 12 Years
work type: Intern
salary range: $56K-$116K
job description: Frontend Web Developers design and implement user interfaces for websites, ensuring they are visually appealing and user-friendly. They collaborate with designers and backend developers to create seamless web experiences for u

In [18]:
def _text_cleaning(text):
  if isinstance(text, pd.Series):
    return text.astype(str).str.lower()
  else:
    text=str(text).lower()
    text=re.sub(r'http\S+|www\S+', '', text)
    text=re.sub(r'\S+@\S+', '', text)
    text=re.sub(r'[^a-z0-9\s\-]', ' ', text)
    text=' '.join(text.split())
    return text


def _clean_exp_col(text):
  nums=re.findall(r'\d+', str(text))
  return f"{nums[0]}-{nums[1]}"

In [19]:
data=data.apply(_text_cleaning)
data['experience']=data['experience'].apply(_clean_exp_col)
data.head()

,job title,skills,experience,work type,salary range,job description
0,digital marketing specialist,"social media platforms (e.g., facebook, twitte...",5-15,intern,$59k-$99k,social media managers oversee an organizations...
1,web developer,"html, css, javascript frontend frameworks (e.g...",2-12,intern,$56k-$116k,frontend web developers design and implement u...
2,operations manager,quality control processes and methodologies st...,0-12,temporary,$61k-$104k,quality control managers establish and enforce...
3,network engineer,wireless network design and architecture wi-fi...,4-11,full-time,$65k-$91k,"wireless network engineers design, implement, ..."
4,event manager,event planning conference logistics budget man...,1-12,intern,$64k-$87k,a conference manager coordinates and manages c...


In [20]:
def _create_data_format(row):
  input_text=f"job_title: {row['job title']} | "\
              f"skills: {row['skills']} | " \
              f"experience: {row['experience']} years | " \
              f"mode: {row['work type']} | " \
              f"salary: {row['salary range']}"
  output_text=row['job description']
  return {
      'input_ids':input_text,
      'labels':output_text
  }

data['t5-data']=data.apply(_create_data_format, axis=1)
data['input_text']=data['t5-data'].apply(lambda x: x['input_ids'])
data['output_text']=data['t5-data'].apply(lambda x: x['labels'])

print(f"Inputs: {data['input_text'].iloc[0]}")
print(f"Outputs: {data['output_text'].iloc[0]}")

Inputs: job_title: digital marketing specialist | skills: social media platforms (e.g., facebook, twitter, instagram) content creation and scheduling social media analytics and insights community engagement paid social advertising | experience: 5-15 years | mode: intern | salary: $59k-$99k
Outputs: social media managers oversee an organizations social media presence. they create and schedule content, engage with followers, and analyze social media metrics to drive brand awareness and engagement.


In [21]:
os.makedirs("data", exist_ok=True)
data.to_csv("data/phase1_data.csv",index=False)

In [22]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch

os.makedirs("t5-model", exist_ok=True)

model_name="t5-base"
model=T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer=T5Tokenizer.from_pretrained(model_name, legacy=False)

model.save_pretrained("./t5-model/t5-base")
tokenizer.save_pretrained("./t5-model/t5-base")

print(f"Model Parameters: {model.num_parameters()}")
print(f"Model Size: {model.num_parameters() / 1e6:.0f}M")

device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Device: {device}")

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Model Parameters: 222903552
Model Size: 223M
Device: cpu


In [23]:
from torch.utils.data import DataLoader, Dataset

df=pd.read_csv('./data/phase1_data.csv')
df.head()

,job title,skills,experience,work type,salary range,job description,t5-data,input_text,output_text
0,digital marketing specialist,"social media platforms (e.g., facebook, twitte...",5-15,intern,$59k-$99k,social media managers oversee an organizations...,{'input_ids': 'job_title: digital marketing sp...,job_title: digital marketing specialist | skil...,social media managers oversee an organizations...
1,web developer,"html, css, javascript frontend frameworks (e.g...",2-12,intern,$56k-$116k,frontend web developers design and implement u...,{'input_ids': 'job_title: web developer | skil...,"job_title: web developer | skills: html, css, ...",frontend web developers design and implement u...
2,operations manager,quality control processes and methodologies st...,0-12,temporary,$61k-$104k,quality control managers establish and enforce...,{'input_ids': 'job_title: operations manager |...,job_title: operations manager | skills: qualit...,quality control managers establish and enforce...
3,network engineer,wireless network design and architecture wi-fi...,4-11,full-time,$65k-$91k,"wireless network engineers design, implement, ...",{'input_ids': 'job_title: network engineer | s...,job_title: network engineer | skills: wireless...,"wireless network engineers design, implement, ..."
4,event manager,event planning conference logistics budget man...,1-12,intern,$64k-$87k,a conference manager coordinates and manages c...,{'input_ids': 'job_title: event manager | skil...,job_title: event manager | skills: event plann...,a conference manager coordinates and manages c...


In [24]:
from sklearn.model_selection import train_test_split
train_df, test_temp=train_test_split(df, test_size=0.15, random_state=42)
test_df, val_df=train_test_split(test_temp, test_size=0.5, random_state=42)
print(f"Train Data Size: {len(train_df)}")
print(f"Test Data Size: {len(test_df)}")
print(f"Validation Data Size: {len(val_df)}")

Train Data Size: 1373549
Test Data Size: 121195
Validation Data Size: 121196


In [25]:
class JDDataset(Dataset):
  def __init__(self,dataframe,tokenizer,max_input_len=128,max_output_len=512):
    self.dataframe=dataframe
    self.tokenizer=tokenizer
    self.max_input_len=max_input_len
    self.max_output_len=max_output_len

  def __len__(self):
    return len(self.dataframe)

  def __getitem__(self,idx):
    row=self.dataframe.iloc[idx]

    input_text=row['input_text']
    input_encoding=self.tokenizer(
        input_text,
        max_length=self.max_input_len,
        padding='max_length',
        truncation=True,
        return_tensors='pt',
    )

    output_text=row['output_text']
    output_encoding=self.tokenizer(
        output_text,
        max_length=self.max_output_len,
        padding='max_length',
        truncation=True,
        return_tensors='pt',
    )

    return {
        'input_ids':input_encoding['input_ids'].squeeze(),
        'attention_mask':input_encoding['attention_mask'].squeeze(),
        'labels':output_encoding['input_ids'].squeeze(),
        'decoder_attention_mask':output_encoding['attention_mask'].squeeze()
    }

tokenizer=T5Tokenizer.from_pretrained("./t5-model/t5-base")

train_dataset=JDDataset(train_df,tokenizer)
test_dataset=JDDataset(test_df,tokenizer)
val_dataset=JDDataset(val_df,tokenizer)

train_loader=DataLoader(train_dataset,batch_size=16,shuffle=True)
test_loader=DataLoader(test_dataset,batch_size=16,shuffle=False)
val_loader=DataLoader(val_dataset,batch_size=16,shuffle=False)

print(f"Train batch: {len(train_loader)}")
print(f"Test batch: {len(test_loader)}")
print(f"Validation batch: {len(val_loader)}")


Train batch: 85847
Test batch: 7575
Validation batch: 7575


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

class ConstraintLoss(nn.Module):
  def __init__(self, tokenizer, weight=0.2):
    super().__init__()
    self.tokenizer=tokenizer
    self.weight=weight

  def _check_constraint(self,text,constraint_type):
    # Hard-coded by Gemini - later add all values from tuple
    constraints = {
        'salary': r'\$|\₹|\d+\s?(?:k|K|LPA|lakhs?)|'
                  r'(?:competitive|salary|ctc|compensation|pay)',

        'work_type': r'intern(?:ship)?|full[-\s]?time|part[-\s]?time|'
                     r'contract|temporary|freelance|consultant|permanent|'
                     r'apprentice|trainee',

        'title': r'developer|engineer|manager|scientist|analyst|architect|'
                 r'designer|consultant|administrator|lead|specialist|'
                 r'director|head|officer|associate|intern|researcher|'
                 r'data\s?scientist|ml\s?engineer|ai\s?engineer|'
                 r'backend|frontend|full[-\s]?stack'
    }
    pattern=constraints.get(constraint_type, "")
    return bool(re.search(pattern, text, re.IGNORECASE)) if pattern else True

  def _forward(self,logits,labels,input_ids):
    ce_loss=nn.CrossEntropyLoss()(logits.view(-1,logits.size(-1)),
                                  labels.view(-1))
    constraint_penalty=0
    for i in range(logits.size(0)):
      decoded=self.tokenizer.decode(logits[i].argmax(dim=-1),
                                    skil_special_tokens=True)
      if not self._check_constraint(decoded,'salary'):
        constraint_penalty+=1.0
      if not self._check_constraint(decoded,'work_type'):
        constraint_penalty+=0.5
      if not self._check_constraint(decoded,'title'):
        constraint_penalty+=0.5

    constrainte_penalty=constraint_penalty/logits.size(0)
    total_loss=ce_loss+self.weight*constraint_penalty
    return total_loss

def _train_epoch(model,train_loader,optimizer,device,constraint_loss_fn):
  model.train()
  total_loss=0

  for batch in tqdm(train_loader,desc='Training'):
    input_ids=batch['input_ids'].to(device)
    attention_mask=batch['attention_mask'].to(device)
    labels=batch['labels'].to(device)

    outputs=model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        labels=labels
    )

    loss=outputs.loss
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    total_loss+=loss.item()

  return total_loss/len(train_loader)

model=T5ForConditionalGeneration.from_pretrained('./t5-model/t5-base')
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

optimizer=optim.AdamW(model.parameters(),lr=5e-5)
constraint_loss_fn=ConstraintLoss(tokenizer,weight=0.2)

num_epochs=3
for epoch in range(num_epochs):
  print(f"Epoch {epoch+1}/{num_epochs}")
  train_loss=_train_epoch(model,train_loader,optimizer,
                          device,constraint_loss_fn)
  print(f"Train Loss: {train_loss}")


Epoch 1/3


Training:   0%|          | 2/85847 [01:57<1383:45:08, 58.03s/it]